# Foundations of AI/ML by IIIT-Hyderabad & Talent Sprint
# Lab08 Case Study 1

## Towards a modern Back Propagation Implementation ##

In [1]:
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
from sklearn import manifold, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_mldata

#Load MNIST datset 
mnist = fetch_mldata('MNIST original')
X, Y = mnist.data, mnist.target
Y = Y.astype(int)

X = X[::10, :]     ## taking the whole data will take a lot of processing time
Y = Y[::10]
# digits = datasets.load_digits(n_class=10)
# # Create our X and y data
# X = digits.data
# Y = digits.target
print(X.shape, Y.shape)
num_examples = X.shape[0]      ## training set size
nn_input_dim = X.shape[1]      ## input layer dimensionality
nn_output_dim = len(np.unique(Y))       ## output layer dimensionality

params = {
    "lr":0.0001,        ## learning_rate
    "max_iter":500,
    "weight_init":"xavier",
    "h_dimn":100,     ## hidden_layer_size
}

(7000, 784) (7000,)


### 1.1 Weight Initializations
Note that we do not know what the final value of every weight should be in the trained network, but with proper data normalization it is reasonable to assume that approximately half of the weights will be positive and half of them will be negative.

Zero Weight Initialization: This turns out to be a mistake, because if every neuron in the network computes the same output, then they will also all compute the same gradients during backpropagation and undergo the exact same parameter updates. In other words, there is no source of asymmetry between neurons if their weights are initialized to be the same.

As a solution, it is common to initialize the weights of the neurons to small numbers (random or unique) and refer to doing so as symmetry breaking. The idea is that the neurons are all random and unique in the beginning, so they will compute distinct updates and integrate themselves as diverse parts of the full network. Instead of using random initializations, it is also possible to use small numbers drawn from a uniform distribution, but this seems to have relatively little impact on the final performance in practice.

It is worth mentioning that if you do not know which technique should be chosen as weight initilalizaion method, Xaiver is often choosen as a initial try.



In [2]:
def xavier_init(fan_in, fan_out):
    ## using FanAvg variation
    n = (fan_in+fan_out)/2
    limit = np.sqrt(3.0 * 1 / n)
    return np.random.uniform(size = (fan_in, fan_out), low = -limit, high = +limit)

def weight_initialization(params):
    hdim = params["h_dimn"]
    winit = params["weight_init"]
    if winit == "random":
        np.random.seed(0)
        W1 = np.random.randn(nn_input_dim, hdim)
        b1 = np.random.randn(1, hdim)
        W2 = np.random.randn(hdim, nn_output_dim)
        b2 = np.random.randn(1, nn_output_dim)
    elif winit == "zeros":
        W1 = np.zeros((nn_input_dim, hdim))
        b1 = np.zeros((1, hdim))
        W2 = np.zeros((hdim, nn_output_dim))
        b2 = np.zeros((1, nn_output_dim))
    elif winit == "xavier":
        W1 = xavier_init(nn_input_dim, hdim)
        b1 = xavier_init(1, hdim)
        W2 = xavier_init(hdim, nn_output_dim)
        b2 = xavier_init(1, nn_output_dim)
    elif winit == "uniform":
        W1 = np.random.uniform(size=(nn_input_dim, hdim), low=-1, high=1)/np.sqrt(nn_input_dim)
        b1 = np.random.uniform(size=(1, hdim), low=-1, high=1)
        W2 = np.random.uniform(size=(hdim, nn_output_dim), low=-1, high=1)/np.sqrt(hdim)
        b2 = np.random.uniform(size=(1, nn_output_dim), low=-1, high=1)
    elif winit == "normal":
        W1 = np.random.normal(loc = 0, scale = 0.5, size = (nn_input_dim, hdim))
        b1 = np.random.normal(loc = 0, scale = 0.5, size=(1, hdim))
        W2 = np.random.normal(loc = 0, scale = 0.5, size = (hdim, nn_output_dim))
        b2 = np.random.normal(loc = 0, scale = 0.5, size=(1, nn_output_dim))
    return W1, b1, W2, b2 

def softmax(x):
    exp_scores = np.exp(x)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return probs

def build_model():
    W1, b1, W2, b2 = weight_initialization(params)
    # This is what we return at the end
    model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
    return model

def feedforward(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    z1 = x.dot(W1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(W2) + b2
    probs = softmax(z2)
    return a1, probs

def backpropagation(model, x, y, a1, probs):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    
    delta3 = probs
    delta3[range(y.shape[0]), y] -= 1
    dW2 = (a1.T).dot(delta3)
    db2 = np.sum(delta3, axis=0, keepdims=True)
    delta2 = delta3.dot(W2.T) * (1 - np.power(a1, 2))
    dW1 = np.dot(x.T, delta2)
    db1 = np.sum(delta2, axis=0)
    return dW2, db2, dW1, db1

def calculate_loss(model, x, y):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    
    # Forward propagation to calculate predictions
    _, probs = feedforward(model, x)
    
    # Calculating the cross entropy loss
    corect_logprobs = -np.log(probs[range(y.shape[0]), y])
    data_loss = np.sum(corect_logprobs)
    
    return 1./y.shape[0] * data_loss

def test(model, x, y):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation to calculate predictions
    _, probs = feedforward(model, x)
    preds = np.argmax(probs, axis=1)
    return np.count_nonzero(y==preds)/y.shape[0]

def train(model, X_train, X_test, Y_train, Y_test, verbose=True):
    # Gradient descent. For each batch...
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    for i in range(0, params["max_iter"]):

        # Forward propagation
        a1, probs = feedforward(model, X_train)

        # Backpropagation
        dW2, db2, dW1, db1 = backpropagation(model, X_train, Y_train, a1, probs)

        # Gradient descent parameter update
        W1 += -params["lr"] * dW1
        b1 += -params["lr"] * db1
        W2 += -params["lr"] * dW2
        b2 += -params["lr"] * db2
        
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        if verbose and i % 50 == 0:
            print("Loss after iteration %i: %f" %(i, calculate_loss(model, X_train, Y_train)),
                  ", Test accuracy:", test(model, X_test, Y_test), "\n")
    return model

### 1.2 Experiment with different Weight Initializations and evaluate the corresponding test accuracies

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4)
t = ["xavier","uniform","normal","zeros","random"]

for i in range(5):
    params["weight_init"] = t[i]
    model = build_model()
    model = train(model, X_train, X_test, Y_train, Y_test, verbose=False)
    print(params, "TestAccuracy=", test(model,X_test, Y_test))
    

{'weight_init': 'xavier', 'lr': 0.0001, 'h_dimn': 100, 'max_iter': 500} TestAccuracy= 0.9160714285714285
{'weight_init': 'uniform', 'lr': 0.0001, 'h_dimn': 100, 'max_iter': 500} TestAccuracy= 0.9232142857142858
{'weight_init': 'normal', 'lr': 0.0001, 'h_dimn': 100, 'max_iter': 500} TestAccuracy= 0.7960714285714285
{'weight_init': 'zeros', 'lr': 0.0001, 'h_dimn': 100, 'max_iter': 500} TestAccuracy= 0.10928571428571429
{'weight_init': 'random', 'lr': 0.0001, 'h_dimn': 100, 'max_iter': 500} TestAccuracy= 0.7164285714285714


### 2. Selecting Hyperparameters
scikit-learn provides a function: GridSearchCV to optimize your neural network's hyper-parameters automatically. We just provide the range or possible value of hyperparameters as the parameter

In [4]:
from sklearn.grid_search import GridSearchCV
from sklearn.neural_network import MLPClassifier
parameters = {'activation' : ["tanh", "relu"],
            'learning_rate_init' : [0.0001, 0.001],
            'hidden_layer_sizes' : [(300,), (300, 100), (100, 50)],
            'solver' : ["adam","sgd"]
             }
clf = MLPClassifier()
clf = GridSearchCV(estimator=clf, param_grid=parameters, verbose=2, cv=2)
clf.fit(X_train, Y_train)   ## might take about 10 minutes depending on number of total parameters


/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Fitting 2 folds for each of 24 candidates, totalling 48 fits
[CV] learning_rate_init=0.0001, solver=adam, activation=tanh, hidden_layer_sizes=(300,) 


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   41.5s remaining:    0.0s


[CV]  learning_rate_init=0.0001, solver=adam, activation=tanh, hidden_layer_sizes=(300,) -  41.5s
[CV] learning_rate_init=0.0001, solver=adam, activation=tanh, hidden_layer_sizes=(300,) 


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

[CV]  learning_rate_init=0.001, solver=sgd, activation=tanh, hidden_layer_sizes=(100, 50) -  19.1s
[CV] learning_rate_init=0.001, solver=sgd, activation=tanh, hidden_layer_sizes=(100, 50) 


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  learning_rate_init=0.001, solver=sgd, activation=tanh, hidden_layer_sizes=(100, 50) -  14.7s
[CV] learning_rate_init=0.0001, solver=adam, activation=relu, hidden_layer_sizes=(300,) 
[CV]  learning_rate_init=0.0001, solver=adam, activation=relu, hidden_layer_sizes=(300,) -   8.6s
[CV] learning_rate_init=0.0001, solver=adam, activation=relu, hidden_layer_sizes=(300,) 
[CV]  learning_rate_init=0.0001, solver=adam, activation=relu, hidden_layer_sizes=(300,) -   9.3s
[CV] learning_rate_init=0.0001, solver=sgd, activation=relu, hidden_layer_sizes=(300,) 
[CV]  learning_rate_init=0.0001, solver=sgd, activation=relu, hidden_layer_sizes=(300,) -   2.9s
[CV] learning_rate_init=0.0001, solver=sgd, activation=relu, hidden_layer_sizes=(300,) 
[CV]  learning_rate_init=0.0001, solver=sgd, activation=relu, hidden_layer_sizes=(300,) -   2.8s
[CV] learning_rate_init=0.001, solver=adam, activation=relu, hidden_layer_sizes=(300,) 
[CV]  learning_rate_init=0.001, solver=adam, activation=relu, hidden_

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed: 13.5min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'hidden_layer_sizes': [(300,), (300, 100), (100, 50)], 'solver': ['adam', 'sgd'], 'activation': ['tanh', 'relu'], 'learning_rate_init': [0.0001, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=2)

In [5]:
print(clf.best_estimator_)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(300,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
